In [1]:
import numpy as np
import cupy as cp
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import interp
from itertools import cycle
from tensorflow.keras import layers, models
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


#  Data Processing

In [2]:
gear_defects = [0, 35, 76, 77]

In [3]:
# Input Image Size
size = [300, 400]

In [4]:
X_train = np.load('gears_train_300x400_0,76,77,35.npy')
X_test = np.load('gears_test_300x400_0,76,77,35.npy')
y_train = np.load('gears_ytrain_300x400_0,76,77,35.npy')
y_test = np.load('gears_ytest_300x400_0,76,77,35.npy')

# Training 

In [5]:
# Define early stopping and best model checkpoint
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, min_delta=0.1, patience=40)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1, save_best_only=True)

In [6]:
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(64, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(128, (5, 5), strides=(1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='RMSprop',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 98, 64)        7808      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 48, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 48, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 23, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 23, 256)       2

In [7]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
262/262 [==============================] - ETA: 0s - loss: 7.6646 - accuracy: 0.8331
Epoch 00001: val_accuracy improved from -inf to 0.83430, saving model to best_model.h5
262/262 [==============================] - 6s 25ms/step - loss: 7.6646 - accuracy: 0.8331 - val_loss: 0.4199 - val_accuracy: 0.8343
Epoch 2/100
262/262 [==============================] - ETA: 0s - loss: 0.3786 - accuracy: 0.9065
Epoch 00002: val_accuracy improved from 0.83430 to 0.91956, saving model to best_model.h5
262/262 [==============================] - 6s 22ms/step - loss: 0.3786 - accuracy: 0.9065 - val_loss: 0.5190 - val_accuracy: 0.9196
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 0.2931 - accuracy: 0.9304
Epoch 00003: val_accuracy improved from 0.91956 to 0.92486, saving model to best_model.h5
262/262 [==============================] - 6s 23ms/step - loss: 0.2931 - accuracy: 0.9304 - val_loss: 0.2714 - val_accuracy: 0.9249
Epoch 4/100
261/262 [=========================

Epoch 29/100
262/262 [==============================] - ETA: 0s - loss: 0.2403 - accuracy: 0.9413
Epoch 00029: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 0.2403 - accuracy: 0.9413 - val_loss: 0.4239 - val_accuracy: 0.9528
Epoch 30/100
262/262 [==============================] - ETA: 0s - loss: 0.2349 - accuracy: 0.9548
Epoch 00030: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 0.2349 - accuracy: 0.9548 - val_loss: 0.1795 - val_accuracy: 0.9542
Epoch 31/100
262/262 [==============================] - ETA: 0s - loss: 0.2664 - accuracy: 0.9370
Epoch 00031: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 0.2664 - accuracy: 0.9370 - val_loss: 0.2599 - val_accuracy: 0.9499
Epoch 32/100
262/262 [==============================] - ETA: 0s - loss: 0.2449 - accuracy: 0.9507
Epoch 00032: val_accuracy did not improve from 

In [8]:
model.load_weights('best_model.h5')

In [9]:
model.save('alexnet_RMSprop.h5')

In [10]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'alexnt_RMSprop_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [11]:
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(64, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(128, (5, 5), strides=(1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer=SGD(learning_rate=0.00001, momentum=0.9),
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 98, 64)        7808      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 48, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 36, 48, 128)       204928    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 17, 23, 128)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 23, 256)      

In [12]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
261/262 [============================>.] - ETA: 0s - loss: 1.3538 - accuracy: 0.7777
Epoch 00001: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 1.3508 - accuracy: 0.7780 - val_loss: 0.7714 - val_accuracy: 0.7577
Epoch 2/100
262/262 [==============================] - ETA: 0s - loss: 0.5188 - accuracy: 0.8131
Epoch 00002: val_accuracy did not improve from 0.97110
262/262 [==============================] - 5s 21ms/step - loss: 0.5188 - accuracy: 0.8131 - val_loss: 0.6744 - val_accuracy: 0.7755
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 0.4721 - accuracy: 0.8293
Epoch 00003: val_accuracy did not improve from 0.97110
262/262 [==============================] - 5s 21ms/step - loss: 0.4721 - accuracy: 0.8293 - val_loss: 0.6194 - val_accuracy: 0.8136
Epoch 4/100
262/262 [==============================] - ETA: 0s - loss: 0.4421 - accuracy: 0.8438
Epoch 00004: val_accuracy did not improve from 0.97

Epoch 30/100
262/262 [==============================] - ETA: 0s - loss: 0.3323 - accuracy: 0.8891
Epoch 00030: val_accuracy did not improve from 0.97110
262/262 [==============================] - 5s 21ms/step - loss: 0.3323 - accuracy: 0.8891 - val_loss: 0.4967 - val_accuracy: 0.7977
Epoch 31/100
262/262 [==============================] - ETA: 0s - loss: 0.3352 - accuracy: 0.8911
Epoch 00031: val_accuracy did not improve from 0.97110
262/262 [==============================] - 5s 21ms/step - loss: 0.3352 - accuracy: 0.8911 - val_loss: 0.4927 - val_accuracy: 0.7977
Epoch 32/100
262/262 [==============================] - ETA: 0s - loss: 0.3298 - accuracy: 0.8902
Epoch 00032: val_accuracy did not improve from 0.97110
262/262 [==============================] - 5s 21ms/step - loss: 0.3298 - accuracy: 0.8902 - val_loss: 0.5168 - val_accuracy: 0.8011
Epoch 33/100
262/262 [==============================] - ETA: 0s - loss: 0.3268 - accuracy: 0.8919
Epoch 00033: val_accuracy did not improve from 

In [15]:
dropout = 0.5
model = models.Sequential()
model.add(layers.Conv2D(64, (11, 11), strides=(4,4), activation='relu', input_shape=(size[0],size[1],1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(128, (5, 5), strides=(1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3, 3), strides=(1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2)))
model.add(layers.Dropout(dropout))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='adadelta',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 73, 98, 64)        7808      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 48, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 36, 48, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 36, 48, 128)       204928    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 23, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 17, 23, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 17, 23, 256)      

In [16]:
# Fitting model with early stopping and checkpoint save
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/100
262/262 [==============================] - ETA: 0s - loss: 17.3087 - accuracy: 0.5760
Epoch 00001: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 17.3087 - accuracy: 0.5760 - val_loss: 2.4527 - val_accuracy: 0.7505
Epoch 2/100
262/262 [==============================] - ETA: 0s - loss: 10.3464 - accuracy: 0.6566
Epoch 00002: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 10.3464 - accuracy: 0.6566 - val_loss: 1.6660 - val_accuracy: 0.7505
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 7.5392 - accuracy: 0.6657
Epoch 00003: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 7.5392 - accuracy: 0.6657 - val_loss: 1.1841 - val_accuracy: 0.7505
Epoch 4/100
262/262 [==============================] - ETA: 0s - loss: 5.8790 - accuracy: 0.6713
Epoch 00004: val_accuracy did not improve from 

Epoch 30/100
262/262 [==============================] - ETA: 0s - loss: 0.4923 - accuracy: 0.8358
Epoch 00030: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 21ms/step - loss: 0.4923 - accuracy: 0.8358 - val_loss: 0.7105 - val_accuracy: 0.7934
Epoch 31/100
262/262 [==============================] - ETA: 0s - loss: 0.4770 - accuracy: 0.8386
Epoch 00031: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 22ms/step - loss: 0.4770 - accuracy: 0.8386 - val_loss: 0.7032 - val_accuracy: 0.7938
Epoch 32/100
262/262 [==============================] - ETA: 0s - loss: 0.4726 - accuracy: 0.8420
Epoch 00032: val_accuracy did not improve from 0.97110
262/262 [==============================] - 6s 21ms/step - loss: 0.4726 - accuracy: 0.8420 - val_loss: 0.6980 - val_accuracy: 0.7948
Epoch 33/100
262/262 [==============================] - ETA: 0s - loss: 0.4573 - accuracy: 0.8438
Epoch 00033: val_accuracy did not improve from 

In [ ]:
model.load_weights('best_model.h5')

In [ ]:
model.save('alexnet_adadelta.h5')

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'alexnt_adadelta_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)